# ASUM

In [1]:
import os
import re

import numpy as np
import pandas as pd

from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from reviews.config import processed_data_dir, asum_input_dir, asum_output_dir
from reviews.models import asum

## Generate Input Data

In [2]:
# load tokens
reviews_df = pd.read_json(processed_data_dir / "preprocessed_reviews.json.gz")

# load vocabulary
vocabulary = np.load(asum_input_dir / "vocabulary.npy", allow_pickle=True).item()

In [3]:
with open(asum_input_dir / "WordList.txt", "w") as f:
    for word in vocabulary:
        f.write(f"{word}\n")

In [4]:
sentence_list = asum_input_dir / "BagOfSentences.txt"

if os.path.exists(sentence_list):
    os.remove(sentence_list)

with open(sentence_list, "a") as f:
    for doc in reviews_df["tokens"]:
        f.write(f"{len(doc)}\n")
        for tokens in doc:
            indexes = " ".join([str(vocabulary[t]) for t in tokens])
            f.write(f"{indexes}\n")

## Run

In [ ]:
alpha = 0.1
beta = [0.001, 0.1, 0]
gamma = [1, 1]
n_topics = 5
iterations = 1000

asum(
    str(alpha),
    "/".join([str(x) for x in beta]),
    "/".join([str(x) for x in gamma]),
    str(n_topics),
    iterations=str(iterations),
)

## Analysis

In [ ]:
filename_prefix = f"STO2-T{n_topics}-S2(2)-A{alpha}-B{','.join([f'{float(x)}' for x in beta])}-G{','.join([f'{float(x):.1f}' for x in gamma])}-I{iterations}"

In [ ]:
df = pd.read_csv(asum_output_dir / f"{filename_prefix}-ProbWords.csv")


def map_topic_sentiment(column_name):
    if "Unnamed" in column_name:
        return None

    values = column_name.split("-")
    return {
        "sentiment": int(values[0][1]),
        "topic": int(values[1][1]),
        "colname": column_name,
    }


columns = df.columns.map(map_topic_sentiment)

In [ ]:
def get_word_weight(text):
    match = re.match("(\w+)\s+\((.*?)\)", text)
    word, weight = match.groups()
    return word, float(weight)


n_topics = 5
n_top_words = 10

fig, axes = plt.subplots(2, n_topics, figsize=(15, 6), sharex="all")
axes = axes.flatten()

for idx, item in enumerate(columns):
    if item is None:
        continue

    ax = axes[idx]
    words = df[item["colname"]][:n_top_words]
    weights = list(map(list, zip(*words.map(get_word_weight))))

    weights = pd.DataFrame({"names": weights[0], "weights": weights[1]})
    weights.sort_values("weights", inplace=True)

    ax.barh(weights["names"], weights["weights"], height=0.5)
    ax.set_title(item["colname"])

    if idx > n_topics - 1:
        ax.set_xlabel(f"Topic {item['topic'] + 1}")

    if idx == 0 or idx == n_topics:
        ax.set_ylabel("Positive" if item["sentiment"] == 0 else "Negative")
fig.tight_layout()

In [ ]:
import seaborn as sns

df1 = pd.DataFrame(list(df["S0-T0"].apply(get_word_weight)), columns=["word", "positive"])
df2 = pd.DataFrame(list(df["S1-T0"].apply(get_word_weight)), columns=["word", "negative"])

df_tot = pd.merge(df1, df2, on="word", how="left")
df_tot.sort_values(by="positive", ascending=False, inplace=True)
df_tot = df_tot[:10]

df_tot.set_index("word", inplace=True)
df_tot = df_tot.rename_axis(columns=["sentiment"]).stack().rename("value")
df_tot = df_tot.reset_index()

df_tot.sort_values(by="value", ascending=False, inplace=True)
df_tot = df_tot[:20]

fig = plt.figure(figsize=(20, 10))
sns.barplot(
    y=df_tot["word"],
    x=df_tot["value"],
    hue=df_tot["sentiment"],
    dodge=True,
    palette=sns.color_palette("terrain"),
)

In [ ]:
import seaborn as sns

pi_df = pd.read_csv(asum_output_dir / f"{filename_prefix}-Pi.csv", header=None)

doc_sentiment = pi_df.idxmax(axis=1).map(lambda x: "positive" if x == 0 else "negative")
doc_sentiment.loc[pi_df[0] == 0.5] = "neutral"

reviews_df["sentiment"] = doc_sentiment

fig, axes = plt.subplots(1, 2, figsize=(10, 3))
sns.histplot(x=doc_sentiment, ax=axes[0])

counts = doc_sentiment.value_counts()
counts.plot(
    ax=axes[1],
    kind="pie",
    colors=sns.color_palette("pastel")[0:7],
    autopct="%.0f%%",
)

In [ ]:
theta_df = pd.read_csv(asum_output_dir / f"{filename_prefix}-Theta.csv")
theta_df.drop(columns=["Unnamed: 10"], inplace=True)

mask = theta_df >= 0.3
mask = mask.apply(lambda x: list(mask.columns[x]), axis=1)

reviews_df["topics"] = mask.apply(lambda x: list(map(map_topic_sentiment, x)))

In [ ]:
reviews_df.to_json(asum_output_dir / "topics.json", orient="records")

In [ ]:
phi_df = pd.read_csv(asum_output_dir / f"{filename_prefix}-Phi.csv")
phi_df.sort_values(by="S1-T1", ascending=False, inplace=True)
phi_df["S1-T1"]

### WordClouds by Topic

In [ ]:
wc = WordCloud(height=400, width=800)
wc.generate_from_frequencies(
    dict(zip(phi_df["Unnamed: 0"].values, phi_df["S1-T4"].values))
)
wc.to_image()

In [ ]:
theta_df

In [ ]:
pi_df

In [ ]:
phi_df.sort_values("S1-T1", ascending=False)

In [ ]:
count = Counter()

for x in reviews_df["topics"].values:
    topics = set(["T" + str(y["topic"]) for y in x])
    count.update(topics)

topics_count = pd.DataFrame(count.items(), columns=["topic", "count"])
topics_count["topic"] = topics_count["topic"].astype("category")

order = topics_count.sort_values(by="count", ascending=False).topic
fig = plt.figure(figsize=(8, 5))
ax = sns.barplot(
    y=topics_count["topic"], x=topics_count["count"], order=order, orient="h"
)
ax.set_title("Topics")
fig.tight_layout()

#### Filtrare i topic-sentiment classificando il sentiment

- T1-S0: 34, T1-S1: 56 ---> S1
- T1-S0: 74, T1-S1: 21 ---> S0
- T1-S0: 67, T1-S1: 67 ---> S2 (neutral?)

In [ ]:
pos_count = Counter()
neg_count = Counter()


for x in reviews_df["topics"].values:
    pos_topics = set(["T" + str(st["topic"]) for st in x if st["sentiment"] == 0])
    neg_topics = set(["T" + str(st["topic"]) for st in x if st["sentiment"] == 1])

    pos_count.update(pos_topics)
    neg_count.update(neg_topics)


pos_df = pd.DataFrame(pos_count.items(), columns=["topic", "pos"])
neg_df = pd.DataFrame(neg_count.items(), columns=["topic", "neg"])

st_counts = pd.merge(pos_df, neg_df, on="topic")
st_counts["topic"] = st_counts["topic"].astype("category")

total = st_counts["pos"] + st_counts["neg"]
st_counts["pos"] = st_counts["pos"] / total * 100
st_counts["neg"] = st_counts["neg"] / total * 100

st_counts.set_index("topic", inplace=True)
st_counts.sort_index(inplace=True)
st_counts = st_counts.iloc[[int(o[1]) for o in order][::-1]]

st_counts.plot(kind="barh", stacked=True, color=["red", "green"])

In [ ]:
prod_df = pd.read_json(processed_data_dir / "meta_computer_components.json.gz")
data_df = pd.merge(left=prod_df, right=reviews_df, on="asin")

# data_df = data_df.loc[data_df["brand"] == "Intel"]
data_df = data_df.loc[data_df["sentiment"] == "positive"]
fig = plt.figure(figsize=(20, 4))
sns.histplot(x=data_df["timestamp"], hue=data_df["sentiment"])

In [ ]:
prod_df = pd.read_json(processed_data_dir / "meta_computer_components.json.gz")
data_df = pd.merge(left=prod_df, right=reviews_df, on="asin")

data_df = data_df.loc[data_df["sentiment"] == "negative"]
fig = plt.figure(figsize=(20, 4))
sns.histplot(x=data_df["timestamp"], hue=data_df["sentiment"])

In [ ]:
prod_df = pd.read_json(processed_data_dir / "meta_computer_components.json.gz")
data_df = pd.merge(left=prod_df, right=reviews_df, on="asin")
data_df = data_df.loc[data_df["brand"].isin(["Intel", "AMD", "Dell", "Asus"])]

fig = plt.figure(figsize=(20, 10))
sns.countplot(x=data_df["brand"], hue=data_df["sentiment"])

In [ ]:
prod_df = pd.read_json(processed_data_dir / "meta_computer_components.json.gz")
data_df = pd.merge(left=prod_df, right=reviews_df, on="asin")

data_df["category"] = data_df["category"].apply(lambda x: x[4] if len(x) > 4 else "other")

fig = plt.figure(figsize=(30, 10))
plt.xticks(rotation=45)
sns.countplot(x=data_df["category"].astype("string"), hue=data_df["sentiment"])

In [ ]:
# topic sentiment nel tempo

In [ ]:
t = pd.crosstab(
    data_df["overall"].apply(
        lambda x: "negative" if x < 3 else ("neutral" if x == 3 else "positive")
    ),
    data_df["sentiment"].astype("category"),
)
t

In [ ]:
import numpy as np

cm = t.to_numpy()
np.sum(cm.diagonal()) / np.sum(cm)